# シンプルなFlask API + Cloudflare Tunnel テスト

最小限のFlask APIサーバーをGoogle Colabで公開する例です。

## 1. 必要なパッケージのインストール

In [ ]:
!pip install -q flask flask-cors
!wget -q https://github.com/cloudflare/cloudflared/releases/latest/download/cloudflared-linux-amd64.deb
!sudo dpkg -i cloudflared-linux-amd64.deb
!rm cloudflared-linux-amd64.deb

print("✅ インストール完了")

## 2. Flask APIサーバーの定義

In [ ]:
from flask import Flask, request, jsonify
from flask_cors import CORS

app = Flask(__name__)
CORS(app)  # CORS対応（重要！）

@app.route("/api", methods=["POST"])
def api():
    data = request.json
    return jsonify({"message": "Hello from Colab!", "data": data})

@app.route("/health", methods=["GET"])
def health():
    return jsonify({"status": "ok", "service": "Simple Flask API"})

print("✅ Flask アプリ定義完了")

## 3. Flaskサーバー + Cloudflare Tunnelの起動

In [ ]:
import threading
import subprocess
import time
import re

# Flaskをバックグラウンドで起動
def run_flask():
    app.run(host='0.0.0.0', port=5000, debug=False)

flask_thread = threading.Thread(target=run_flask, daemon=True)
flask_thread.start()

# Flaskが起動するまで待機
time.sleep(3)
print("✅ Flaskサーバー起動完了")

# Cloudflare Tunnelを起動
print("\n🌐 Cloudflare Tunnelを起動中...\n")

cloudflared_process = subprocess.Popen(
    ['cloudflared', 'tunnel', '--url', 'http://localhost:5000'],
    stdout=subprocess.PIPE,
    stderr=subprocess.STDOUT,
    universal_newlines=True
)

# Cloudflare URLを取得
public_url = None
for line in cloudflared_process.stdout:
    print(line.strip())
    
    # URLを抽出
    if 'trycloudflare.com' in line:
        match = re.search(r'https://[a-zA-Z0-9-]+\.trycloudflare\.com', line)
        if match:
            public_url = match.group(0)
            break

if public_url:
    print("\n" + "="*60)
    print("🎉 APIサーバー起動完了！")
    print("="*60)
    print(f"\n📡 公開URL: {public_url}")
    print(f"\n✅ ヘルスチェック:")
    print(f"   curl {public_url}/health")
    print(f"\n✅ APIテスト:")
    print(f"   curl -X POST {public_url}/api \\")
    print(f"     -H 'Content-Type: application/json' \\")
    print(f"     -d '{{\"test\": \"data\"}}'")
    print("\n" + "="*60 + "\n")
    
    # サーバーを稼働し続ける
    try:
        cloudflared_process.wait()
    except KeyboardInterrupt:
        print("\n⏹️  サーバーを停止しました")
        cloudflared_process.terminate()
else:
    print("\n⚠️ Cloudflare URLの取得に失敗しました")

## 4. テスト（別のセルで実行）

In [ ]:
import requests

# 上で表示された公開URLを設定
API_URL = "YOUR_CLOUDFLARE_URL"  # 例: https://xxxx.trycloudflare.com

if API_URL != "YOUR_CLOUDFLARE_URL":
    # ヘルスチェック
    print("🔍 ヘルスチェック...")
    response = requests.get(f"{API_URL}/health")
    print(f"Status: {response.status_code}")
    print(f"Response: {response.json()}")
    
    print("\n🔍 APIテスト...")
    # APIテスト
    response = requests.post(
        f"{API_URL}/api",
        json={"test": "data", "message": "Hello from client!"}
    )
    print(f"Status: {response.status_code}")
    print(f"Response: {response.json()}")
else:
    print("⚠️ API_URLを設定してください")